In [12]:
import numpy as np
import cv2 as cv
from keras.models import load_model


model = load_model("model/incep_asl.h5")

def getcnthull(mask_img):
    contours, hierarchy = cv.findContours(mask_img, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    contours = max(contours, key=lambda x: cv.contourArea(x))
    hull = cv.convexHull(contours)
    return contours, hull

In [13]:
k = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
label_dict = {}
for i in range(0, 26):
    label_dict[i] = k[i]
label_dict[26] = 'del'
label_dict[27] = 'Space'
label_dict[28] = 'Nothing'
label_dict

{0: 'A',
 1: 'B',
 2: 'C',
 3: 'D',
 4: 'E',
 5: 'F',
 6: 'G',
 7: 'H',
 8: 'I',
 9: 'J',
 10: 'K',
 11: 'L',
 12: 'M',
 13: 'N',
 14: 'O',
 15: 'P',
 16: 'Q',
 17: 'R',
 18: 'S',
 19: 'T',
 20: 'U',
 21: 'V',
 22: 'W',
 23: 'X',
 24: 'Y',
 25: 'Z',
 26: 'del',
 27: 'Space',
 28: 'Nothing'}

In [ ]:
label_dict

In [19]:
cap = cv.VideoCapture(0)  
while cap.isOpened():
    _, img = cap.read()
       
    hsvim = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    lower = np.array([0, 48, 80])
    upper = np.array([20, 255, 255])
    skinMask = cv.inRange(hsvim, lower, upper)
        
    blurred = cv.blur(skinMask, (2, 2))

    ret, thresh = cv.threshold(blurred, 0, 255, cv.THRESH_BINARY)

    contours, hull = getcnthull(thresh)
    cnt = contours

    x, y, w, h = cv.boundingRect(cnt)
    img = cv.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        

#     img_g = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    hand_img = img[x-100:x+w+80, y-100:y+h+100]
    print(hand_img.shape)
    hand_img = cv.flip(hand_img, 1)
    resized = cv.resize(hand_img, (200, 200))
    normalized = resized / 255.0
    reshaped = np.reshape(normalized, (1, 200, 200, 3))
#     probab = model.predict(reshaped)
    
    probab = np.array(model.predict(reshaped))

    #     result = model.predict(reshaped)
    
    idx = probab.argmax()
    
    # print(result[0])
    
    print(label_dict[idx])
        
    # img = cv.rectangle(img,(x,y-40),(x+w,y),label_dict[result[0]],-1)
    # img = cv.putText(img, label_dict[result[0]], (x+10, y+10),cv.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    
    cv.putText(img, label_dict[idx], (x+10, y-10),cv.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    
    
    cv.imshow("LIVE", img)
               
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

(297, 451, 3)
W
(341, 445, 3)
A
(289, 391, 3)
T
(320, 433, 3)
T
(273, 411, 3)
I
(263, 293, 3)
N
(302, 338, 3)
T
(264, 298, 3)
T
(256, 393, 3)
Space
(256, 359, 3)
Space
(276, 311, 3)
Space
(275, 343, 3)
W
(255, 251, 3)
Space
(245, 271, 3)
M
(244, 275, 3)
T
(251, 284, 3)
Space
(273, 286, 3)
Space
(294, 285, 3)
Space
(233, 282, 3)
M
(267, 282, 3)
Space
(270, 313, 3)
Y
(271, 355, 3)
B
(224, 296, 3)
Space
(289, 348, 3)
Space
(266, 350, 3)
M
(273, 321, 3)
Space
(233, 278, 3)
U
(235, 283, 3)
Space
(230, 293, 3)
Space
(231, 282, 3)
Space
(229, 273, 3)
Space
(218, 270, 3)
Space
(225, 264, 3)
Space
(227, 262, 3)
Space
(219, 271, 3)
Space
(227, 274, 3)
Space
(217, 269, 3)
Space
(217, 273, 3)
Space
(233, 292, 3)
Space
(231, 274, 3)
A
(237, 268, 3)
Space
(225, 263, 3)
Space
(241, 287, 3)
Space
(243, 275, 3)
M
(233, 273, 3)
A
(233, 271, 3)
Space
(232, 266, 3)
Space
(204, 307, 3)
E
(223, 260, 3)
Space
(200, 340, 3)
Y
(206, 339, 3)
T
(208, 310, 3)
T
(211, 331, 3)
A
(205, 340, 3)
A
(204, 378, 3)
E
(200

In [ ]:
img = cv.imread("sign_3.jpg")
hsvim = cv.cvtColor(img, cv.COLOR_BGR2HSV)
lower = np.array([0, 48, 80])
upper = np.array([20, 255, 255])
skinMask = cv.inRange(hsvim, lower, upper)
# skin = cv.bitwise_and(img, img, mask = skinMask)
    
    
    
blurred = cv.blur(skinMask, (2, 2))
# cv.imshow("img", skin)    
    
ret, thresh = cv.threshold(blurred, 0, 255, cv.THRESH_BINARY)

contours, hull = getcnthull(thresh)
cnt = contours

x, y, w, h = cv.boundingRect(cnt)
img = cv.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
cv.imshow("img", img)
# print(img.shape)

# img_g = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
hand_img = img[x-100:x+w+100, y-100:y+h+100]
print(hand_img.shape)
hand_img = cv.flip(hand_img, 1)
resized = cv.resize(hand_img, (200, 200))
normalized = resized / 255.0
reshaped = np.reshape(normalized, (1, 200, 200, 3))
        
#     print(hand_img.shape)
# resized = cv.resize(hand_img, (28, 28))
# normalized = resized / 255.0
# reshaped = np.reshape(normalized, (1, 28, 28, 1))
# cv.imshow("Resh",reshaped[0])
result = model.predict(reshaped)
idx = np.argmax(result)
# # print(result[0])
print(label_dict[idx])
# # img = cv.rectangle(img,(x,y-40),(x+w,y),label_dict[result[0]],-1)
# # img = cv.putText(img, label_dict[result[0]], (x+10, y+10),cv.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
# cv.putText(img, label_dict[idx], (x+10, y-10),cv.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
# cv.imshow("FInal", img)
        
cv.waitKey()


cv.destroyAllWindows()

In [23]:
import cv2

IMAGE_SIZE = 200
CROP_SIZE = 400


classes = label_dict
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame.
    ret, img = cap.read()
    hsvim = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    lower = np.array([0, 48, 80])
    upper = np.array([20, 255, 255])
    skinMask = cv.inRange(hsvim, lower, upper)
        
    blurred = cv.blur(skinMask, (2, 2))

    ret, thresh = cv.threshold(blurred, 0, 255, cv.THRESH_BINARY)

    contours, hull = getcnthull(thresh)
    cnt = contours

    x, y, w, h = cv.boundingRect(cnt)
    frame = cv.rectangle(img, (x-100, y-100), (x + w + 100, y + h + 100), (0, 255, 0), 2)

#     frame = cv2.flip(frame, 1)
    # Preprocessing the frame before input to the model.
    cropped = frame[0:CROP_SIZE, 0:CROP_SIZE]
    resized_frame = cv2.resize(cropped, (200, 200))
    reshaped_frame = (np.array(resized_frame)).reshape((1, 200, 200, 3))
#     Normalizing
    frame_for_model = reshaped_frame/255

    # Predicting the frame.
    prediction = np.array(model.predict(frame_for_model))
    predicted_class = classes[prediction.argmax()]      # Selecting the max confidence index.
    print(predicted_class)

    cv2.putText(frame, predicted_class , (10, 450), 1, 2, (255, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('frame', frame)

    # Press q to quit
    k = cv2.waitKey(1) & 0xFF
    if k == ord('q'):
        break

# When everything done, release the capture.
cap.release()
cv2.destroyAllWindows()

X
Space
N
N
N
W
N
N
N
S
N
N
S
S
S
W
W
S
T
T
N
W
T
N
W
M
M
M
N
F
T
T
N
F
N
N
N
N
N
S
S
S
S
M
S
S
M
M
M
S
M
F
F
W
W
W
W
W
W
T
W
T
Y
Y
Y
W
Y
Y
Y
W
Y
T
Y
Y
Y
E
E
Y
W
N
W
Y
N
W
N
Y
Y
T
E
N
Y
Y
N
E
E
E
E
E
E
E
E
E
E
E
B
E
W
E
W
B
W
B
W
W
W
W
W
W
Y
Y
Y
W
W
S
W
W
S
S
S
E
Y
Y
Y
W
Y
W
W
W
W
T
W
Y
W
T
W
W
W
W
W
T
T
W
T
T
T
W
T
T
T
W
T
W
T
T
T
T
T
T
T
T
T
W
T
T
W
T
M
W
W
W
W
W
W
W
W
M
Space
W
W
S
W
W
W
Space
Space
Space
Space
Space
P
T
O
T
T
T
M
A
M
S
S
T


In [ ]:
cv.imshow("KK",img)
cv.waitKey(1)